In [1]:
import pandas as pd
import pandana as pdna
from pandana.network import Network
from urbansim.utils import misc

In [2]:
with pd.HDFStore('./2015_09_01_bayarea_v3.h5', mode='r') as store:
    parcels = store.parcels

In [3]:
parcels.head()

,development_type_id,land_value,acres,county_id,zone_id,proportion_undevelopable,tax_exempt_status,apn,parcel_id_local,geom_id,imputation_flag,x,y,shape_area
parcel_id,,,,,,,,,,,,,,
229116,1,0.0,3.360520,1,725,0.0,0,099 029001700,,10305106092872,_,-121.795620,37.655379,1.359956e+04
244166,1,0.0,1.294423,1,715,0.0,0,099B540210200,,11107351665227,_,-121.713004,37.717277,5.238348e+03
202378,25,6036500.0,14.993605,1,820,0.0,1,085A643106000,,11030175960628,merged,-122.014199,37.655260,6.067702e+04
2004420,-1,179954.0,316.247146,97,1401,0.0,0,141-100-012,,6381677629073,"_, nr_zone_yrblt, bsqft_from_nrsqft, stories1",-122.771868,38.727893,1.279808e+06
340332,2,0.0,0.621275,1,763,0.0,0,525 166004800,,314875459798,_,-121.974508,37.546277,2.514215e+03


In [4]:
pg = pd.read_csv('02_01_2016_parcels_geography.csv')

In [5]:
pg.head()

,geom_id,jurisdiction_id,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1


In [6]:
pg = pg.merge(parcels[['geom_id','x','y']], left_on='geom_id', right_on='geom_id')

In [7]:
pg.head()

,geom_id,jurisdiction_id,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea,x,y
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,-121.795620,37.655379
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,-121.713004,37.717277
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0,-122.014199,37.655260
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0,-122.771868,38.727893
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1,-121.974508,37.546277


In [8]:
with pd.HDFStore('2015_06_01_osm_bayarea4326.h5') as store:
    edges = store.edges
    nodes = store.nodes

In [9]:
alts = pd.read_csv('bart_alternatives.csv')

In [10]:
alts.head()

,alternative,station,x,y
0,1,Fillmore,-122.433081,37.784604
1,1,Van Ness,-122.421647,37.785524
2,1,Union Square,-122.409734,37.787277
3,1,3rd St Mission,-122.400657,37.784987
4,1,Ballpark,-122.392733,37.779053


In [11]:
net = Network(node_x=nodes['x'], 
              node_y=nodes['y'], 
              edge_from=edges['from'], 
              edge_to=edges['to'], 
              edge_weights=edges[['weight']])

In [12]:
pg['node_id'] = net.get_node_ids(pg.x, pg.y)

In [13]:
net.init_pois(num_categories=1, max_dist=3000, max_pois=1)
net.set_pois("tmp", alts[alts.alternative == 1].x, alts[alts.alternative == 1].y)
nearest = net.nearest_pois(3000, "tmp", num_pois=1)

In [18]:
nearest.columns = ['dist']

In [20]:
pg = pg.merge(nearest, how='left', left_on='node_id', right_index=True)

In [22]:
pg['within'] = pg.apply(lambda row: 1 if row.dist < 3000.0 else 0, axis=1)

In [26]:
len(pg[pg.within == 1])

48810

In [25]:
len(pg)

1956208

In [29]:
pg.tpp_id = pg.apply(lambda row: 'bart1' if row.within == 1 else row.tpp_id, axis=1)

In [32]:
pg.drop(['x', 'y', 'node_id', 'dist', 'within'], axis=1, inplace=True)

In [33]:
pg.to_csv('')

,geom_id,jurisdiction_id,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1
5,8421216428506,26000,NaN,b2,NaN,NaN,26000NAb2NANA,1,1
6,858019333320,57792,NaN,NaN,NaN,NaN,57792NANANANA,1,0
7,4014739544205,20018,NaN,NaN,NaN,NaN,20018NANANANA,1,0
8,14651032512384,13,NaN,NaN,NaN,NaN,00013NANANANA,0,0
9,10084939736689,8142,NaN,NaN,NaN,NaN,08142NANANANA,1,0
